In [45]:
# define path constants
landing_data = '../data/landing/'
raw_data = '../data/raw/'
curated_data = '../data/curated/'
analysis_data = '../data/analysis/'

In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/12 16:27:05 WARN Utils: Your hostname, PVM203L resolves to a loopback address: 127.0.1.1; using 172.20.203.188 instead (on interface eth0)
23/08/12 16:27:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/12 16:27:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [60]:
from pyspark.sql.functions import col, lit

def prepare_raw_tlc_data(source: str, rename_cols: dict[str:str]) -> None:
    '''
    Retrieves downloaded TLC data (landing data), combining separate monthly data
    into one dataframe, and renames and filters columnsof interest (). 
    Saves this raw data down accordingly
    Arguments:
        source = TLC data source (yellow, green, or fhvhv)
        rename_cols = dict with keys = original column name, values = new column name
    Ouput: None
    '''
    # get landing data
    all_data = spark.read.parquet(landing_data+source+'/')
    # select columns of interest
    all_data = all_data.select(*rename_cols.keys())
    # rename columns
    all_data = all_data.select([col(c).alias(rename_cols.get(c, c)) for c in all_data.columns])
    # adding an identifier for data source, as curated data will combine all sourcse
    # into one dataframe
    all_data=all_data.withColumn('source',lit(source))
    # save the raw data
    all_data.write.mode('overwrite').parquet(raw_data+source)

In [61]:
# prepare the raw data for each of the yellow, green, and fhvhv data sets in turn
rename_cols = {'tpep_pickup_datetime': 'pickup_datetime', 'tpep_dropoff_datetime': 'dropoff_datetime',
               'trip_distance': 'trip_distance', 'PULocationID':'pickup_id','fare_amount':'fare'}
prepare_raw_tlc_data('yellow', rename_cols)

rename_cols = {'lpep_pickup_datetime': 'pickup_datetime', 'lpep_dropoff_datetime': 'dropoff_datetime',
               'trip_distance': 'trip_distance', 'PULocationID':'pickup_id','fare_amount':'fare'}
prepare_raw_tlc_data('green', rename_cols)

rename_cols = {'pickup_datetime': 'pickup_datetime', 'dropoff_datetime': 'dropoff_datetime',
               'trip_miles': 'trip_distance', 'PULocationID':'pickup_id','base_passenger_fare':'fare'}
prepare_raw_tlc_data('fhvhv', rename_cols)

In [46]:
yellow_all = spark.read.parquet(landing_data+'yellow/')

In [26]:
green_all = spark.read.parquet('../data/green/')

In [27]:
green_all.limit(5)

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
2,2023-05-01 00:52:10,2023-05-01 01:05:26,N,1,244,213,1,6.99,28.9,1.0,0.5,0.0,0.0,null,1.0,31.4,1,1,0.0
2,2023-05-01 00:29:49,2023-05-01 00:50:11,N,1,33,100,1,6.6,30.3,1.0,0.5,5.0,0.0,null,1.0,40.55,1,1,2.75
2,2023-05-01 00:25:19,2023-05-01 00:32:12,N,1,244,244,1,1.34,9.3,1.0,0.5,2.36,0.0,null,1.0,14.16,1,1,0.0
2,2023-05-01 00:07:06,2023-05-01 00:27:33,N,5,82,75,1,7.79,22.73,0.0,0.0,2.29,6.55,null,1.0,32.57,1,1,0.0
2,2023-05-01 00:43:31,2023-05-01 00:46:59,N,1,69,169,1,0.7,6.5,1.0,0.5,0.0,0.0,null,1.0,9.0,2,1,0.0


In [4]:
fhvhv_all = spark.read.parquet('../data/fhvhv/')

In [5]:
select_cols=('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance','PULocationID', 'fare_amount')
yellow_all = yellow_all.select(*select_cols)
rename_cols = {'tpep_pickup_datetime': 'pickup_datetime', 'tpep_dropoff_datetime': 'dropoff_datetime',
               'trip_distance': 'trip_distance', 'PULocationID':'pickup_id','fare_amount':'fare'}
yellow_all = yellow_all.select([col(c).alias(rename_cols.get(c, c)) for c in yellow_all.columns])

In [28]:
yellow_all=yellow_all.withColumn('trip_time', yellow_all.dropoff_datetime - yellow_all.pickup_datetime)

In [43]:
yellow_all.withColumn('trip_time', F.col('trip_time').cast('long')/60.round(4))

SyntaxError: invalid decimal literal (2969704009.py, line 1)

In [41]:
yellow_all.limit(5)

pickup_datetime,dropoff_datetime,trip_distance,pickup_id,fare,trip_time
2023-05-01 00:33:13,2023-05-01 00:53:01,7.8,138,33.8,INTERVAL '0 00:19...
2023-05-01 00:42:49,2023-05-01 01:11:18,8.1,138,35.9,INTERVAL '0 00:28...
2023-05-01 00:56:34,2023-05-01 01:13:39,9.1,138,35.2,INTERVAL '0 00:17...
2023-05-01 00:00:52,2023-05-01 00:20:12,8.21,138,33.1,INTERVAL '0 00:19...
2023-05-01 00:05:50,2023-05-01 00:19:41,7.9,138,31.0,INTERVAL '0 00:13...


In [31]:
yellow_all.select('pickup_datetime').cast(long)-yellow_all.select('dropoff_datetime').cast(long)

AttributeError: 'DataFrame' object has no attribute 'cast'

In [25]:
# check date range
yellow_all = yellow_all.where(
    # != 0 is fine, but let's also take into account some more realistic filters
    (F.col('fare') <= 0) 
    | (F.col('trip_distance') <= 0)
).withColumn('After_discount', df.Course_Fees - df.Discount)

pickup_datetime,dropoff_datetime,trip_distance,pickup_id,fare
2023-05-01 00:14:01,2023-05-01 00:14:23,0.0,132,3.0
2023-05-01 00:30:22,2023-05-01 00:36:54,1.06,249,-8.6
2023-05-01 00:22:53,2023-05-01 00:23:02,0.0,132,3.0
2023-05-01 00:40:04,2023-05-01 00:40:24,0.0,265,150.0
2023-05-01 00:18:15,2023-05-01 00:18:19,0.0,70,20.0
2023-05-01 00:58:55,2023-05-01 01:17:02,10.86,138,-42.2
2023-05-01 00:07:06,2023-05-01 00:07:15,0.0,79,70.0
2023-05-01 00:31:42,2023-05-01 00:32:23,0.0,265,78.0
2023-05-01 00:30:33,2023-05-01 00:30:54,0.0,138,3.0
2023-05-01 00:41:56,2023-05-01 00:42:07,0.0,132,0.0


In [14]:
select_cols=('lpep_pickup_datetime', 'lpep_dropoff_datetime', 'trip_distance','PULocationID', 'fare_amount')
green_all = green_all.select(*select_cols)
rename_cols = {'lpep_pickup_datetime': 'pickup_datetime', 'lpep_dropoff_datetime': 'dropoff_datetime',
               'trip_distance': 'trip_distance', 'PULocationID':'pickup_id','fare_amount':'fare'}
green_all = green_all.select([col(c).alias(rename_cols.get(c, c)) for c in green_all.columns])

In [15]:
green_all.limit(5)

pickup_datetime,dropoff_datetime,trip_distance,pickup_id,fare
2023-05-01 00:52:10,2023-05-01 01:05:26,6.99,244,28.9
2023-05-01 00:29:49,2023-05-01 00:50:11,6.6,33,30.3
2023-05-01 00:25:19,2023-05-01 00:32:12,1.34,244,9.3
2023-05-01 00:07:06,2023-05-01 00:27:33,7.79,82,22.73
2023-05-01 00:43:31,2023-05-01 00:46:59,0.7,69,6.5


In [18]:
select_cols=('pickup_datetime', 'dropoff_datetime', 'trip_miles','PULocationID', 'base_passenger_fare')
fhvhv_all = fhvhv_all.select(*select_cols)
rename_cols = {'pickup_datetime': 'pickup_datetime', 'dropoff_datetime': 'dropoff_datetime',
               'trip_miles': 'trip_distance', 'PULocationID':'pickup_id','base_passenger_fare':'fare'}
fhvhv_all = fhvhv_all.select([col(c).alias(rename_cols.get(c, c)) for c in fhvhv_all.columns])

In [19]:
fhvhv_all.limit(5)

pickup_datetime,dropoff_datetime,trip_distance,pickup_id,fare
2023-05-01 00:09:39,2023-05-01 00:21:04,2.247,32,12.3
2023-05-01 00:43:35,2023-05-01 00:51:48,1.826,81,8.85
2023-05-01 00:06:54,2023-05-01 00:20:24,3.88,121,12.7
2023-05-01 00:24:27,2023-05-01 00:36:01,2.31,92,11.59
2023-05-01 00:39:13,2023-05-01 00:57:07,9.45,53,27.69


In [16]:
fhvhv_all.limit(10)

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
HV0005,B03406,null,2023-04-30 23:59:28,null,2023-05-01 00:09:39,2023-05-01 00:21:04,32,81,2.247,685,12.3,0.0,0.37,1.09,0.0,0.0,0.0,10.27,N,N,N,N,Y
HV0005,B03406,null,2023-05-01 00:37:36,null,2023-05-01 00:43:35,2023-05-01 00:51:48,81,254,1.826,493,8.85,0.0,0.27,0.78,0.0,0.0,0.0,7.75,N,N,N,N,Y
HV0003,B03404,B03404,2023-05-01 00:00:50,2023-05-01 00:06:41,2023-05-01 00:06:54,2023-05-01 00:20:24,121,92,3.88,810,12.7,0.0,0.38,1.13,0.0,0.0,1.0,12.71,N,N,,N,N
HV0003,B03404,B03404,2023-05-01 00:19:08,2023-05-01 00:23:57,2023-05-01 00:24:27,2023-05-01 00:36:01,92,53,2.31,694,11.59,0.0,0.35,1.03,0.0,0.0,0.0,10.57,N,N,,N,N
HV0003,B03404,B03404,2023-05-01 00:34:25,2023-05-01 00:37:41,2023-05-01 00:39:13,2023-05-01 00:57:07,53,122,9.45,1074,27.69,0.0,0.83,2.46,0.0,0.0,0.0,24.27,Y,N,,N,N
HV0003,B03404,B03404,2023-05-01 00:27:29,2023-05-01 00:28:56,2023-05-01 00:29:14,2023-05-01 00:38:00,234,114,1.65,526,11.83,0.0,0.35,1.05,2.75,0.0,0.0,7.12,N,N,,N,N
HV0003,B03404,B03404,2023-05-01 00:17:02,2023-05-01 00:45:09,2023-05-01 00:46:20,2023-05-01 01:17:46,113,258,11.57,1886,38.63,6.55,1.36,4.01,2.75,0.0,0.0,32.93,N,N,,N,N
HV0003,B03404,B03404,2023-04-30 23:57:59,2023-04-30 23:59:11,2023-05-01 00:00:59,2023-05-01 00:06:22,243,127,0.9,323,8.85,0.0,0.27,0.79,0.0,0.0,0.0,5.38,N,N,,N,N
HV0003,B03404,B03404,2023-05-01 00:23:10,2023-05-01 00:24:39,2023-05-01 00:26:18,2023-05-01 00:33:50,166,238,1.39,452,8.69,0.0,0.26,0.77,0.75,0.0,0.0,6.08,Y,N,,N,N
HV0003,B03404,B03404,2023-05-01 00:58:07,2023-05-01 00:59:42,2023-05-01 00:59:55,2023-05-01 01:09:42,164,143,2.77,587,13.6,0.0,0.41,1.21,2.75,0.0,1.0,9.16,N,N,,N,N
